In [42]:
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid, save_image
from PIL import Image
import numpy as np
from torch.amp import autocast, GradScaler

In [44]:
# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [46]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import os

In [48]:
class CustomDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label, subdir in enumerate(['tumor', 'notumor']):
            subdir_path = os.path.join(img_dir, subdir)
            for img_name in os.listdir(subdir_path):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    self.image_paths.append(os.path.join(subdir_path, img_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert('RGB')  # Converts grayscale to 3-channel RGB safely

        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Ensure 1 channel
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # For 1 channel
])


# Create dataset and dataloader
dataset = CustomDataset('data/generation_data/Training', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [49]:
# Generator Model
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels=1, feature_g=64):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            # latent_dim x 1 x 1 → 4x4
            nn.ConvTranspose2d(latent_dim, feature_g * 16, 4, 1, 0, bias=False),
            nn.BatchNorm2d(feature_g * 16),
            nn.ReLU(True),
        
            # 4x4 → 8x8
            nn.ConvTranspose2d(feature_g * 16, feature_g * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 8),
            nn.ReLU(True),
        
            # 8x8 → 16x16
            nn.ConvTranspose2d(feature_g * 8, feature_g * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 4),
            nn.ReLU(True),
        
            # 16x16 → 32x32
            nn.ConvTranspose2d(feature_g * 4, feature_g * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g * 2),
            nn.ReLU(True),
        
            # 32x32 → 64x64
            nn.ConvTranspose2d(feature_g * 2, feature_g, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g),
            nn.ReLU(True),
        
            # 64x64 → 128x128
            nn.ConvTranspose2d(feature_g, feature_g // 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_g // 2),
            nn.ReLU(True),
        
            # 128x128 → 256x256
            nn.ConvTranspose2d(feature_g // 2, img_channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )


    def forward(self, x):
        return self.gen(x)

In [52]:
class Discriminator(nn.Module):
    def __init__(self, image_channels=1):
        super(Discriminator, self).__init__()
        
        # Define the layers for the discriminator
        self.model = nn.Sequential(
            # 1st convolutional layer
            nn.Conv2d(image_channels, 64, kernel_size=4, stride=2, padding=1),  # Output: (64, 128, 128)
            nn.LeakyReLU(0.2, inplace=True),
            
            # 2nd convolutional layer
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # Output: (128, 64, 64)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 3rd convolutional layer
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),  # Output: (256, 32, 32)
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 4th convolutional layer
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),  # Output: (512, 16, 16)
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            
            # 5th convolutional layer
            nn.Conv2d(512, 1024, kernel_size=4, stride=2, padding=1),  # Output: (1024, 8, 8)
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            
            # Flatten the output and pass it through a fully connected layer
            nn.Flatten(),
            nn.Linear(1024 * 8 * 8, 1),  # Output: Scalar
            # nn.Sigmoid()  # Sigmoid to output probabilities
        )

    def forward(self, x):
        return self.model(x)

In [54]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()

latent_dim=100
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

optimizer_g = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_d = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999))

# Create output dir
os.makedirs("generated", exist_ok=True)

# Fixed noise for image generation visualization
fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

num_epochs = 100

In [55]:
scaler_G = GradScaler()
scaler_D = GradScaler()

In [64]:
for epoch in range(num_epochs):
    loop = tqdm(dataloader, desc=f"Epoch [{epoch+1}/{num_epochs}]", leave=False)

    for i, (real_imgs, _) in enumerate(loop):
        real_imgs = real_imgs.to(device)
        b = real_imgs.size(0)

        real_labels = torch.full((b,), 0.9, device=device)  # Label smoothing
        fake_labels = torch.zeros(b, device=device)

        noise = torch.randn(b, latent_dim, 1, 1, device=device)

        # ---------------------
        # Train Discriminator
        # ---------------------
        optimizer_d.zero_grad()
        with autocast(device_type='cuda'):
            real_pred = discriminator(real_imgs + 0.05 * torch.randn_like(real_imgs)).view(-1)
            d_real_loss = criterion(real_pred, real_labels)

            fake_imgs = generator(noise)
            fake_pred = discriminator(fake_imgs.detach() + 0.05 * torch.randn_like(fake_imgs)).view(-1)
            d_fake_loss = criterion(fake_pred, fake_labels)

            d_loss = d_real_loss + d_fake_loss

        scaler_D.scale(d_loss).backward()
        scaler_D.step(optimizer_d)
        scaler_D.update()

        # ---------------------
        # Train Generator
        # ---------------------
        optimizer_g.zero_grad()
        with autocast(device_type='cuda'):
            gen_labels = torch.ones(b, device=device)
            fake_pred = discriminator(fake_imgs).view(-1)
            g_loss = criterion(fake_pred, gen_labels)

        scaler_G.scale(g_loss).backward()
        scaler_G.step(optimizer_g)
        scaler_G.update()

        loop.set_postfix(D_Loss=f"{d_loss.item():.4f}", G_Loss=f"{g_loss.item():.4f}")

    # ---------------------
    # Save Generated Samples
    # ---------------------
    if (epoch + 1) % 5 == 0 or epoch == 0:
        generator.eval()
        with torch.no_grad():
            fake = generator(fixed_noise).detach().cpu()
            grid = make_grid(fake, nrow=8, normalize=True)
            save_image(grid, f"generated/epoch_{epoch+1:03d}.png")
        generator.train()

    # Save model checkpoint
    torch.save(generator.state_dict(), f"checkpoints/generator_epoch_{epoch+1}.pth")